In [12]:
from utils import *

model = {"P":False, "Q":True, "L":True, "R":True}

alpha = "(P|Q) | F"
expr(alpha)
print(a.args[0].op)

ModuleNotFoundError: No module named 'utils'

In [1]:
def PL_True(alpha, model):
  expr = utils.expr(alpha)
  if expr.op is None: 
    return model[alpha] # single var or constant 
  elif expr.op == "|": 
    # Recursively evaluate each argument and return True if any are True
    return PL_True(expr.args[0], model) or PL_True(expr.args[1], model)
  elif expr.op == "&": 
    # Recursively evaluate each argument and return True if both are True
    return PL_True(expr.args[0], model) and PL_True(expr.args[1], model) 
  elif expr.op == "~": 
    # Recursively evaluate the argument and return the negation
    return not PL_True(expr.args[0], model) 
  elif expr.op == '->':
        # p -> q is the same as ~p | q
        return not PL_True(expr.args[0], model) or PL_True(expr.args[1], model)
  else: 
     raise ValueError("Unsupported operator")
  
# Example usage
model = {'P':False, 'Q':True, 'L':True, 'R':True, 'F':False}
alpha = "(P|Q) | F"
result = PL_True(alpha, model)
print(result)

NameError: name 'utils' is not defined